# Analýza a vizualizace dat

- Analýza a vizualizace dat 
    - Nástroje pro platformu Python
    - Pandas
    - Matplotlib
    - Další nástroje (numpy, scikit, ...)

## Úvodní cvičení

### Příprava dat (pro Excel)

> **Zadání**
> 
> Vytvořte homogenní strukturu, na základě které vytvoříte kontingenční tabulku (např. v Excelu) počítající sumu hodin
> - po učebních skupinách v kombinaci s typem lekce
> - po vyučujících v kombinaci s typem lekce
> - ...


Zadání napovídá, že bude potřeba homogenní struktura (tabulka). Je nutné vědět, co to je kontingenční tabulka (pivot table).

> **Povinné video**
>
> [Základy kontingenční tabulky | EduTV 10 min](https://www.youtube.com/watch?v=IWse9RPLDgM)

Zadání současně říká, jaká data musí být přítomna.
- učební skupina
- typ lekce
- délka lekce
- vyučující
- ...
- předmět

Výchozími daty je json soubor, se kterým jsme již dříve pracovali (aplikace rozvrh hodin).


> **Mikroúkol**
>
> Napište funkci pro spojení textů v listu s možností určení oddělovače

In [ ]:
def joinStr(delimiter, strList):
    result = ''
    
    for index, item in enumerate(strList):
        if index == 0:
            result = item
        else:
            result = delimiter + item
    #
    #
    #
    
    return result

print(joinStr(', ', ['A', 'B', 'C']))

> **Řešení**

In [37]:
def joinStr(delimiter, strList):
    result = ''
    for index, item in enumerate(strList):
        if index == 0:
            result = item
        else:
            result = result + delimiter + item
    return result

print(joinStr(', ', ['A', 'B', 'C']))

A, B, C


In [38]:
import json

with open('rozvrh/data.json', encoding='utf-8') as inputFile:
    data = json.load(inputFile)

In [40]:
print(data.keys())

dict_keys(['events', 'occurrencesDays', 'timeslots', 'colors', 'subjects', 'departments', 'groups', 'entryYears', 'students', 'teachers', 'classrooms', 'areas', 'categories', 'spheres', 'lessonForms'])


In [39]:
for index, item in enumerate(data['events']):
    print(item)
    if index > 2:
        break

{'id': '2D2B1C90-F4FF-11EB-9B74-520D00000000', 'typeId': 'F00CB650-9768-11EB-87D2-030800000000', 'startTime': {'hours': 14, 'minutes': 30}, 'endTime': {'hours': 16, 'minutes': 0}, 'dateCode': '2021-11-15', 'date': {'day': 15, 'month': 11, 'year': 2021}, 'categoryId': 1, 'subjectId': 32269, 'subjectName': 'Letecké elektronické systémy II', 'departmentId': 441, 'departmentName': 'FVT-K206', 'topic': '3. Syntéza kmitočtu ', 'topicId': 62645, 'masterId': '62645/2', 'timeslotsId': 1, 'timeslotsName': 'BR', 'lessonOrder': 8, 'lessonFormId': 2, 'lessonFormName': 'CV', 'lessonUnit': 45, 'lessonsCount': 2, 'groupsIds': ['A8CFD4D0-8CA4-11EB-BA48-520D00000000'], 'groupsNames': ['23-3LT-AV-C'], 'groupsEntryYearsIds': [], 'classroomsIds': [331], 'classroomsNames': ['Č1/120'], 'classroomsAreasIds': [2], 'teachersIds': [540], 'teachersNames': ['Němeček, Jiří'], 'teachersDepartmentsIds': [441], 'isLocked': True}
{'id': 'C65A0170-F4FE-11EB-9B74-520D00000000', 'typeId': 'EFAE42A0-9768-11EB-87D2-03080000

Na základě výpisu připravíme mapovací funkci

> **Připomenutí**
> 
> Využití generátorů

In [41]:
def updateData(data):
    for item in data:
        yield item + 1
    
inputData = [1, 2, 3, 4]
resultData = list(updateData(inputData))
print(resultData)

[2, 3, 4, 5]


Implementace funkce

In [42]:
def dataExtractor(events):
    for item in events:
        resultItem = {
            'id': item['id'],
            'subjectName': item['subjectName'] if 'subjectName' in item else 'unknown',
            'departmentName': item['departmentName'] if 'departmentName' in item else 'unknown',
            'topic': item['topic'] if 'topic' in item else 'unknown',
            'lessonFormName': item['lessonFormName'] if 'lessonFormName' in item else 'unknown',
            'lessonsCount': item['lessonsCount'] if 'lessonsCount' in item else 0,
            'groups': joinStr(', ', item['groupsNames']) if 'groupsNames' in item else 'unknown',
        }
        for teacher in item['teachersNames']:
            newResult = {
                **resultItem, 
                'teacher': teacher
            }
            yield newResult
    pass

Podrobnější rozbor

In [48]:
import json

with open('rozvrh/data.json', encoding='utf-8') as inputFile:
    data = json.load(inputFile)
    
for index, item in enumerate(data['events']):
    if index > 2:
        break
    resultItem = {
        'id': item['id'],
        'subjectName': item['subjectName'] if 'subjectName' in item else 'unknown',
        'departmentName': item['departmentName'] if 'departmentName' in item else 'unknown',
        'topic': item['topic'] if 'topic' in item else 'unknown',
        'lessonFormName': item['lessonFormName'] if 'lessonFormName' in item else 'unknown',
        'lessonsCount': item['lessonsCount'] if 'lessonsCount' in item else 0,
        'groups': joinStr(', ', item['groupsNames']) if 'groupsNames' in item else 'unknown'
    } 
    for teacher in item['teachersNames']:
        finalResult = {
            **resultItem,
            'teacher': joinStr(', ', item['teachersNames']) if 'teachersNames' in item else 'unknown'
        }
        print(finalResult)


In [49]:
import json

with open('rozvrh/data.json', encoding='utf-8') as inputFile:
    data = json.load(inputFile)
    
dataForPivotTable = list(dataExtractor(data['events']))
for index, item in enumerate(dataForPivotTable):
    print(item)
    if index > 2:
        break
        

{'id': '2D2B1C90-F4FF-11EB-9B74-520D00000000', 'subjectName': 'Letecké elektronické systémy II', 'departmentName': 'FVT-K206', 'topic': '3. Syntéza kmitočtu ', 'lessonFormName': 'CV', 'lessonsCount': 2, 'groups': '23-3LT-AV-C', 'teacher': 'Němeček, Jiří'}
{'id': 'C65A0170-F4FE-11EB-9B74-520D00000000', 'subjectName': 'Letecké elektronické systémy II', 'departmentName': 'FVT-K206', 'topic': '1. Letecké rádiové přijímače', 'lessonFormName': 'LAB', 'lessonsCount': 2, 'groups': '23-3LT-AV-C', 'teacher': 'Němeček, Jiří'}
{'id': '396CC6C0-F4FF-11EB-9B74-520D00000000', 'subjectName': 'Letecké elektronické systémy II', 'departmentName': 'FVT-K206', 'topic': '3. Syntéza kmitočtu ', 'lessonFormName': 'LAB', 'lessonsCount': 2, 'groups': '23-3LT-AV-C', 'teacher': 'Němeček, Jiří'}
{'id': '7B7F24E0-F4FF-11EB-9B74-520D00000000', 'subjectName': 'Letecké elektronické systémy II', 'departmentName': 'FVT-K206', 'topic': '5. Úhloměrné navigační systémy NDB, VOR, DVOR, TACAN, VORTAC ', 'lessonFormName': 'CV

In [58]:
def getTotal(teacherName, lessonFormName):
    total = 0
    for item in dataForPivotTable:
        if not (item['teacher'] == teacherName):
            continue
        if not (item['lessonFormName'] == lessonFormName):
            continue
        # obě podmínky jsou splněny
        total = total + item['lessonsCount']
    return total

teacherName = 'Štefek, Alexandr'
lessonFormName = 'CV'

print(teacherName, 'CV', getTotal(teacherName, 'CV'))
print(teacherName, 'P', getTotal(teacherName, 'P'))
print(teacherName, 'S', getTotal(teacherName, 'S'))
print(teacherName, 'LC', getTotal(teacherName, 'LC'))

In [60]:
def getTotals(teacherName, lessonFormNames):
    result = {}
    for item in lessonFormNames:
        result[item] = getTotal(teacherName, item)
    return result

teacherName = 'Štefek, Alexandr'
lessonFormNames = ['CV','P','S', 'LC']
print(teacherName, getTotals(teacherName, lessonFormNames))

Štefek, Alexandr {'CV': 2, 'P': 62, 'S': 0, 'LC': 0}


### Kontingenční tabulka

Kontingenční tabulka je tabulka, kde v buňce je zobrazen výsledek agregační funkce aplikované na vektor hodnot, který byl získán z dat filtrací podle filtrů určených sloupcem i řádkem.

In [71]:
rowFilters = {
    'Štefek, A.': lambda item: item['teacher'] == 'Štefek, Alexandr',
    'Ráčil, T.': lambda item: item['teacher'] == 'Ráčil, Tomáš',   
}

colFilters = {
    'P': lambda item: item['lessonFormName'] == 'P',
    'LC': lambda item: item['lessonFormName'] == 'LC',
    'CV': lambda item: item['lessonFormName'] == 'CV',
}

def doubleFilter(filterA, filterB, data):
    for item in data:
        if not filterA(item):
            continue
        if not filterB(item):
            continue
        yield item

for rowIndex, (rowFilterName, rowFilter) in enumerate(rowFilters.items()):
    if rowIndex == 0:
        print('\t', end='')
        for colIndex, (colFilterName, colFilter) in enumerate(colFilters.items()):
            print('\t', colFilterName, end='')
        print('')
    print(rowFilterName, '\t', end='')
    for colIndex, (colFilterName, colFilter) in enumerate(colFilters.items()):
        vectorData = doubleFilter(rowFilter, colFilter, dataForPivotTable)
        print(len(list(vectorData)), '\t', end='')
    print()

		 P	 LC	 CV
Štefek, A. 	19 	0 	1 	
Ráčil, T. 	8 	0 	0 	


> **Zadání**
>
> Vytvořte funkci, která vrací `filters` pro libovolnou položku

```
 filters = {
    'P': lambda item: item['lessonFormName'] == 'P',
    'LC': lambda item: item['lessonFormName'] == 'LC',
    'CV': lambda item: item['lessonFormName'] == 'CV',
}
```


In [ ]:
def createFilters(data, itemName):
    result = {}
    #
    # pro vsechny polozky v data:
        # jestlize NECO:
            # neco ## pozor, tady je zaludnost lambda nebude stacit PROC?
    #
    return result

> **Řešení**

In [75]:
def createFilter(itemName, itemValue):
    return lambda c: c[itemName] == itemValue

def createFilters(data, itemName):
    result = {}
    for row in data:
        itemValue = row[itemName]
        if itemValue in result:
            continue
        result[itemValue] = createFilter(itemName, itemValue)
    return result

filters = createFilters(dataForPivotTable, 'lessonFormName')
for key, value in filters.items():
    print(key, '\t', value)

CV 	 <function createFilter.<locals>.<lambda> at 0x7f9be26699d0>
LAB 	 <function createFilter.<locals>.<lambda> at 0x7f9be2669160>
unknown 	 <function createFilter.<locals>.<lambda> at 0x7f9be26690d0>
P 	 <function createFilter.<locals>.<lambda> at 0x7f9be2669040>
SEM 	 <function createFilter.<locals>.<lambda> at 0x7f9be3385f70>
PV 	 <function createFilter.<locals>.<lambda> at 0x7f9be33859d0>
EX 	 <function createFilter.<locals>.<lambda> at 0x7f9be33858b0>
PX 	 <function createFilter.<locals>.<lambda> at 0x7f9be3385820>
KON 	 <function createFilter.<locals>.<lambda> at 0x7f9be3385790>
TER 	 <function createFilter.<locals>.<lambda> at 0x7f9be3790700>
SMP 	 <function createFilter.<locals>.<lambda> at 0x7f9be4a94b80>
SZK 	 <function createFilter.<locals>.<lambda> at 0x7f9be516d9d0>
J 	 <function createFilter.<locals>.<lambda> at 0x7f9be3385700>
KRZ 	 <function createFilter.<locals>.<lambda> at 0x7f9be3385670>
STŽ 	 <function createFilter.<locals>.<lambda> at 0x7f9be3385550>
ZK 	 <function

Otestujte si své řešení!

In [77]:
filters = createFilters(dataForPivotTable, 'lessonFormName') 
filters['CV']({'lessonFormName': 'CV'})

True

Využijte své řešení pro generování filtrů pro jinou položku

In [78]:
depFilters = createFilters(dataForPivotTable, 'departmentName') 
print(depFilters.keys())

dict_keys(['FVT-K206', 'unknown', 'FVL-K102', 'FVL-K111', 'FVT-K205', 'FVT-K202', 'FVT-K216', 'FVZ-K301', 'FVZ-K308', 'FVL-K105', 'FVT-K201', 'FVT-K209', 'CBVSS', 'FVT-K207', 'FVT-K217', 'FVT-K208', 'FVL-K109', 'FVL-K108', 'FVL-K101', 'CJV', 'FVT-K215', 'CTVS', 'FVL-K110', 'FVL-K104', 'FVT-K210', 'FVZ-K302', 'FVT-K203', 'ÚOPZHN', 'FVL-K107', 'FVZ-K306', 'FVZ-K305', 'FVZ-K304', 'FVZ-K307', 'FVZ-K303', 'FVT-K211'])


> **Zadání (na doma :)**
>
> Vytvořte funkci která vygeneruje kontingenční tabulku

In [ ]:
def kontingencniTabulka(dataRecords, cellItemName, rowItemName, colItemName, aggregationFunc):
    # vytvor filtry pro radky
    # vytvor filtry pro sloupce
    
    # pro vsechny radkove filtry
        # je-li radek prvnim (s nultym indexem)
            # neco
            # neco
        # pro vsechny sloupcove filtry
            # je-li sloupec prvnim (s nultym indexem)
                # yield
            # filtruj dataRecords na podminky rowFilter & colFilter
            # agreguj data ze ziskaneho vektoru
            # yield agregovanou hodnotu

> **Zadání (na doma :) II**
>
> Upravte funkci do tvaru, níže
>
> Uveďte důvody pro využití této definice

In [ ]:
def kontingencniTabulkaDefinition(cellItemName, rowItemName, colItemName, aggregationFunc):
    # vytvor filtry pro radky
    # vytvor filtry pro sloupce

    def create(dataRecords):

        # pro vsechny radkove filtry
            # je-li radek prvnim (s nultym indexem)
                # neco
                # neco
            # pro vsechny sloupcove filtry
                # je-li sloupec prvnim (s nultym indexem)
                    # yield
                # filtruj dataRecords na podminky rowFilter & colFilter
                # agreguj data ze ziskaneho vektoru
                # yield agregovanou hodnotu
    return create

## Pandas

Pandas je jedna z největších knihoven pro Python. Jedná se o opensource! 
Dokumentaci lze nalézt [zde](https://pandas.pydata.org/).

> **Doporučené video**
>
> [Complete Python Pandas Data Science Tutorial! 1h](https://www.youtube.com/watch?v=vmEHCJofslg)


### DataFrame

Jedním ze základních prvků je [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html).

Zobrazíme si získaná data v tabulkové podobě.

In [61]:
import json
import pandas as pd

with open('rozvrh/data.json', encoding='utf-8') as inputFile:
    data = json.load(inputFile)
    
df = pd.DataFrame(dataExtractor(data['events']))
df

,id,subjectName,departmentName,topic,lessonFormName,lessonsCount,groups,teacher
0,2D2B1C90-F4FF-11EB-9B74-520D00000000,Letecké elektronické systémy II,FVT-K206,3. Syntéza kmitočtu,CV,2,23-3LT-AV-C,"Němeček, Jiří"
1,C65A0170-F4FE-11EB-9B74-520D00000000,Letecké elektronické systémy II,FVT-K206,1. Letecké rádiové přijímače,LAB,2,23-3LT-AV-C,"Němeček, Jiří"
2,396CC6C0-F4FF-11EB-9B74-520D00000000,Letecké elektronické systémy II,FVT-K206,3. Syntéza kmitočtu,LAB,2,23-3LT-AV-C,"Němeček, Jiří"
3,7B7F24E0-F4FF-11EB-9B74-520D00000000,Letecké elektronické systémy II,FVT-K206,"5. Úhloměrné navigační systémy NDB, VOR, DVOR,...",CV,2,23-3LT-AV-C,"Andrle, Miloš"
4,04E49860-F4FF-11EB-9B74-520D00000000,Letecké elektronické systémy II,FVT-K206,2. Letecké rádiové vysílače,LAB,2,23-3LT-AV-C,"Němeček, Jiří"
...,...,...,...,...,...,...,...,...
69258,0750ACE9-323D-4E14-B3CC-ABCDF66F96C2,unknown,FVL-K105,Válečné studie,ZK,0,"5D, 5DVZS, 5CH, 5MIZ, 5MT, 5PZ, 5ŘFZ, 5ŘLZ, 5V...","Staněk, Jaroslav"
69259,DFB9C331-9C9C-4485-B06F-ACCB19E4757E,unknown,FVL-K102,Management,ZK,0,"12-5ŘPOS1, 12-5ŘPOS2, 12-5ŘPOS3, 12-5ŘPOS4, 12...","Richter, Jiří"
69260,3292D722-AC2B-49B8-8E48-5B4EFF2084A2,unknown,FVT-K217,Základy elektrotechniky,ZK,0,"22-3GMOB-C, 22-3LT-C, 22-3TOMO-C, 22-3ZM-C LAB...","Kohl, Zdeněk"
69261,89D2BBB0-F582-4B6C-B8DB-D1D7D0F789CA,unknown,FVT-K217,Základy elektrotechniky,ZK,0,"22-3GMOB-C, 22-3LT-C, 22-3TOMO-C, 22-3ZM-C LAB...","Kohl, Zdeněk"


### Projekce

#### Zobrazení jednoho sloupce

In [85]:
df['subjectName']

0        Letecké elektronické systémy II
1        Letecké elektronické systémy II
2        Letecké elektronické systémy II
3        Letecké elektronické systémy II
4        Letecké elektronické systémy II
                      ...               
69258                            unknown
69259                            unknown
69260                            unknown
69261                            unknown
69262                            unknown
Name: subjectName, Length: 69263, dtype: object

In [124]:
df.iloc[0:-1, [1, 2, 3]]

,subjectName,departmentName,topic
0,Letecké elektronické systémy II,FVT-K206,3. Syntéza kmitočtu
1,Letecké elektronické systémy II,FVT-K206,1. Letecké rádiové přijímače
2,Letecké elektronické systémy II,FVT-K206,3. Syntéza kmitočtu
3,Letecké elektronické systémy II,FVT-K206,"5. Úhloměrné navigační systémy NDB, VOR, DVOR,..."
4,Letecké elektronické systémy II,FVT-K206,2. Letecké rádiové vysílače
...,...,...,...
69257,unknown,FVL-K105,Válečné studie
69258,unknown,FVL-K105,Válečné studie
69259,unknown,FVL-K102,Management
69260,unknown,FVT-K217,Základy elektrotechniky


#### Zobrazení "vybraných sloupců". Indexem dataframe je list názvů sloupců.

In [82]:
df[['id', 'subjectName']]

,id,subjectName
0,2D2B1C90-F4FF-11EB-9B74-520D00000000,Letecké elektronické systémy II
1,C65A0170-F4FE-11EB-9B74-520D00000000,Letecké elektronické systémy II
2,396CC6C0-F4FF-11EB-9B74-520D00000000,Letecké elektronické systémy II
3,7B7F24E0-F4FF-11EB-9B74-520D00000000,Letecké elektronické systémy II
4,04E49860-F4FF-11EB-9B74-520D00000000,Letecké elektronické systémy II
...,...,...
69258,0750ACE9-323D-4E14-B3CC-ABCDF66F96C2,unknown
69259,DFB9C331-9C9C-4485-B06F-ACCB19E4757E,unknown
69260,3292D722-AC2B-49B8-8E48-5B4EFF2084A2,unknown
69261,89D2BBB0-F582-4B6C-B8DB-D1D7D0F789CA,unknown


#### Vytvoření nového sloupce

In [88]:
df['duration'] = df['lessonsCount'] * 45
df

,id,subjectName,departmentName,topic,lessonFormName,lessonsCount,groups,teacher,duration
0,2D2B1C90-F4FF-11EB-9B74-520D00000000,Letecké elektronické systémy II,FVT-K206,3. Syntéza kmitočtu,CV,2,23-3LT-AV-C,"Němeček, Jiří",90
1,C65A0170-F4FE-11EB-9B74-520D00000000,Letecké elektronické systémy II,FVT-K206,1. Letecké rádiové přijímače,LAB,2,23-3LT-AV-C,"Němeček, Jiří",90
2,396CC6C0-F4FF-11EB-9B74-520D00000000,Letecké elektronické systémy II,FVT-K206,3. Syntéza kmitočtu,LAB,2,23-3LT-AV-C,"Němeček, Jiří",90
3,7B7F24E0-F4FF-11EB-9B74-520D00000000,Letecké elektronické systémy II,FVT-K206,"5. Úhloměrné navigační systémy NDB, VOR, DVOR,...",CV,2,23-3LT-AV-C,"Andrle, Miloš",90
4,04E49860-F4FF-11EB-9B74-520D00000000,Letecké elektronické systémy II,FVT-K206,2. Letecké rádiové vysílače,LAB,2,23-3LT-AV-C,"Němeček, Jiří",90
...,...,...,...,...,...,...,...,...,...
69258,0750ACE9-323D-4E14-B3CC-ABCDF66F96C2,unknown,FVL-K105,Válečné studie,ZK,0,"5D, 5DVZS, 5CH, 5MIZ, 5MT, 5PZ, 5ŘFZ, 5ŘLZ, 5V...","Staněk, Jaroslav",0
69259,DFB9C331-9C9C-4485-B06F-ACCB19E4757E,unknown,FVL-K102,Management,ZK,0,"12-5ŘPOS1, 12-5ŘPOS2, 12-5ŘPOS3, 12-5ŘPOS4, 12...","Richter, Jiří",0
69260,3292D722-AC2B-49B8-8E48-5B4EFF2084A2,unknown,FVT-K217,Základy elektrotechniky,ZK,0,"22-3GMOB-C, 22-3LT-C, 22-3TOMO-C, 22-3ZM-C LAB...","Kohl, Zdeněk",0
69261,89D2BBB0-F582-4B6C-B8DB-D1D7D0F789CA,unknown,FVT-K217,Základy elektrotechniky,ZK,0,"22-3GMOB-C, 22-3LT-C, 22-3TOMO-C, 22-3ZM-C LAB...","Kohl, Zdeněk",0


In [89]:
type(df['duration'])

pandas.core.series.Series

In [91]:
for index, item in enumerate(df['duration']):
    print(index, item)
    if index > 2:
        break

0 90
1 90
2 90
3 90


#### Uživatelsky definovaná funkce

In [94]:
def userFunc(pdSeries):
    result = []
    for item in pdSeries:
        result.append(item[4:])
    return result

df['departmentCode'] = userFunc(df['departmentName'])
df['departmentCode']

0        K206
1        K206
2        K206
3        K206
4        K206
         ... 
69258    K105
69259    K102
69260    K217
69261    K217
69262    K217
Name: departmentCode, Length: 69263, dtype: object

In [93]:
df['departmentCode'] = list(map(lambda item: item[4:], df['departmentName']))
df['departmentCode']

0        K206
1        K206
2        K206
3        K206
4        K206
         ... 
69258    K105
69259    K102
69260    K217
69261    K217
69262    K217
Name: departmentCode, Length: 69263, dtype: object

In [95]:
df['departmentCode'] = [item[4:] for item in df['departmentName']]
df['departmentCode']

0        K206
1        K206
2        K206
3        K206
4        K206
         ... 
69258    K105
69259    K102
69260    K217
69261    K217
69262    K217
Name: departmentCode, Length: 69263, dtype: object

### Selekce

#### Přímý index

In [113]:
df.loc[[2]]

,id,subjectName,departmentName,topic,lessonFormName,lessonsCount,groups,teacher,duration,departmentCode
2,396CC6C0-F4FF-11EB-9B74-520D00000000,Letecké elektronické systémy II,FVT-K206,3. Syntéza kmitočtu,LAB,2,23-3LT-AV-C,"Němeček, Jiří",90,K206


In [114]:
df.loc[range(2,7)]

,id,subjectName,departmentName,topic,lessonFormName,lessonsCount,groups,teacher,duration,departmentCode
2,396CC6C0-F4FF-11EB-9B74-520D00000000,Letecké elektronické systémy II,FVT-K206,3. Syntéza kmitočtu,LAB,2,23-3LT-AV-C,"Němeček, Jiří",90,K206
3,7B7F24E0-F4FF-11EB-9B74-520D00000000,Letecké elektronické systémy II,FVT-K206,"5. Úhloměrné navigační systémy NDB, VOR, DVOR,...",CV,2,23-3LT-AV-C,"Andrle, Miloš",90,K206
4,04E49860-F4FF-11EB-9B74-520D00000000,Letecké elektronické systémy II,FVT-K206,2. Letecké rádiové vysílače,LAB,2,23-3LT-AV-C,"Němeček, Jiří",90,K206
5,9A434300-1B6C-11EC-916D-030800000000,unknown,unknown,unknown,unknown,0,,"Leuchter, Jan",0,own
6,9A3BF000-1B6C-11EC-916D-030800000000,unknown,unknown,unknown,unknown,0,,"Leuchter, Jan",0,own


#### Podmínka

Všimněte si přímé komparace vybraného sloupce s výrazem

In [84]:
df['lessonsCount'] > 0

0         True
1         True
2         True
3         True
4         True
         ...  
69258    False
69259    False
69260    False
69261    False
69262    False
Name: lessonsCount, Length: 69263, dtype: bool

Zobrazení vybraných řádků, parametrem je vektor logických proměnných.

In [86]:
df[df['lessonsCount'] > 0]

,id,subjectName,departmentName,topic,lessonFormName,lessonsCount,groups,teacher
0,2D2B1C90-F4FF-11EB-9B74-520D00000000,Letecké elektronické systémy II,FVT-K206,3. Syntéza kmitočtu,CV,2,23-3LT-AV-C,"Němeček, Jiří"
1,C65A0170-F4FE-11EB-9B74-520D00000000,Letecké elektronické systémy II,FVT-K206,1. Letecké rádiové přijímače,LAB,2,23-3LT-AV-C,"Němeček, Jiří"
2,396CC6C0-F4FF-11EB-9B74-520D00000000,Letecké elektronické systémy II,FVT-K206,3. Syntéza kmitočtu,LAB,2,23-3LT-AV-C,"Němeček, Jiří"
3,7B7F24E0-F4FF-11EB-9B74-520D00000000,Letecké elektronické systémy II,FVT-K206,"5. Úhloměrné navigační systémy NDB, VOR, DVOR,...",CV,2,23-3LT-AV-C,"Andrle, Miloš"
4,04E49860-F4FF-11EB-9B74-520D00000000,Letecké elektronické systémy II,FVT-K206,2. Letecké rádiové vysílače,LAB,2,23-3LT-AV-C,"Němeček, Jiří"
...,...,...,...,...,...,...,...,...
69246,84ECDED0-0B09-11EC-8109-E50100000000,Pravděpodobnost a statistika,FVL-K101,14. Aplikace statistických metod v bezpečnostn...,CV,2,12-5ŘPOS2,"Lacinová, Veronika"
69247,E579D940-0B06-11EC-8109-E50100000000,Pravděpodobnost a statistika,FVL-K101,2. Popisná statistika,CV,2,12-5ŘPOS2,"Lacinová, Veronika"
69248,AE5220D0-0B06-11EC-8109-E50100000000,Pravděpodobnost a statistika,FVL-K101,1. Základní statistické pojmy,CV,2,12-5ŘPOS2,"Lacinová, Veronika"
69249,57EA4410-0B07-11EC-8109-E50100000000,Pravděpodobnost a statistika,FVL-K101,"4. Pravděpodobnost – 2. část (podmíněná pst, p...",CV,2,12-5ŘPOS2,"Lacinová, Veronika"


#### Uživatelsky definovaná funkce

### Pivot Table

In [64]:
import numpy as np
pd.pivot_table(df, values='lessonsCount', index=['departmentName'], columns=['lessonFormName'], aggfunc=np.sum, fill_value='')

lessonFormName,CV,EX,J,KOL,KON,KRZ,LAB,P,PV,PX,SEM,SMP,STŽ,SZK,TER,ZK,unknown
departmentName,,,,,,,,,,,,,,,,,
CBVSS,2966.0,,2.0,,5462.0,,266.0,1798.0,,,3748.0,157.0,,,144.0,0.0,
CJV,219344.0,,,,,,,96.0,,,80.0,,,,,,
CTVS,1195152.0,,,,,,,848.0,109850.0,,,,,,,0.0,
FVL-K101,458.0,,,,,,,618.0,,,34.0,,,,,0.0,
FVL-K102,1390.0,,,,,,,1160.0,6400.0,,1105.0,,,0.0,,0.0,
FVL-K104,1038.0,,,,,,,659.0,,,1358.0,,,,,,
FVL-K105,470.0,82.0,,,,,,3318.0,,12872.0,1840.0,,,,270.0,0.0,
FVL-K107,445.0,,,,,,,364.0,5360.0,,22.0,,,0.0,,,
FVL-K108,268.0,,,,,,,120.0,5367.0,,96.0,,,,,0.0,


### Vizualizace

## Matplotlib

### Intro

### Liniové grafy

### Datové grafy

## Další nástroje